# Nhóm 14 Master:  CodeBert base + Distilbert baseline

**Thành viên:**

19120212 - Vũ Công Duy

19120301 - Võ Thành Nam

19120389 - Tô Gia Thuận

19120459 - Hồ Anh Bình


**Lưu ý**: Trên thực tế, để có thể tối ưu hóa điểm số đạt được và tối ưu hóa được thời gian cho phép sử dụng GPU trên Kaggle để huấn luyện, nhóm quyết định chia bài làm thành 3 file notebook để thực hiện. Nhưng trong yêu cầu bài nộp chỉ giới hạn việc nộp 1 file ipynb cộng thêm việc không thể nộp kèm theo file model đã train sẵn. Nên trong bài nộp này nhóm tổng hợp lại cả 3 file notebook thành 1 file notebook duy nhất.

# Bài toán

Mục tiêu của bài toán là sắp xếp chính xác thứ tự của các cell trong file, với thông tin cho trước là giữa những code cell đã đảm bảo thứ tự đúng. Dễ hiểu hơn, ta có ví dụ sau.

Một file chưa có thứ tự đúng:
```
code_1
code_2
code_3
markdown_1
markdown_2
```
và một file đã có thứ tự đúng:
```
code_1
markdown_2
code_2
code_3
markdown_1
```
Các markdown cell có thể ở bất kì thứ tự nào, nhưng giữa các code cell thì sẽ không có trường hợp `code_2` ở trước `code_1`.

# Ý tưởng chính

Vị trí của một cell trong notebook sẽ được biễu diễn bằng tham số `rank`.  
Nếu cell đó là cell đầu tiên của notebook thì `rank` mang giá trị 1, cell phía dưới thì `rank` là 2,...   
`rank_pct` chính là tham số chuẩn hoá cho `rank` với mục đích giới hạn giá trị này luôn dưới 1 bằng cách lấy `rank` chia cho tổng số cell của notebook.  
**Output** của mạng neural cũng là `rank_pct` của mỗi cell.  
**Input** sẽ được giải thích ở bên dưới.  

# CodeBert

`code-bert` là một mạng BERT với đầu vào gồm 1 chuỗi ngôn ngữ tự nhiên và 1 chuỗi ngôn ngữ lập trình.  
Chuỗi ngôn ngữ tự nhiên trong trường hợp này sẽ là dữ liệu của file notebook đang xét và chuỗi ngôn ngữ lập trình chính là dữ liệu của tất cả các cell code trong file notebook. Cấu trúc sẽ là :  
`<markdown><s><code1><s><code2><s>....<code_n>`  với `<s>` là kí tự dùng để phân cách.   
Trong quá trình làm, bọn em sẽ có vài thao tác tiền xử lý để tối ưu như : giới hạn độ dài chuỗi markdown, giới hạn độ dài chuỗi code bằng cách chọn random có quy luật,...  

## Setup

In [ ]:
#from dataset import *
from typing import List
import json
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import sparse
from random import sample
from importlib.resources import path

import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import sys, os

import argparse
from bisect import bisect
import gc
import nltk 
from nltk.corpus import stopwords
import re

import torch.nn.functional as F
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup

# Metric 


In [ ]:
def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):
        j = bisect(sorted_so_far, u)
        inversions += i - j
        sorted_so_far.insert(j, u)
    return inversions


def kendall_tau(ground_truth, predictions):
    total_inversions = 0
    total_2max = 0  # twice the maximum possible inversions across all instances
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max

## Config

In [ ]:
data_dir = Path('..//input/AI4Code') # đường dẫn đến input
BATCH_SIZE = 16 #kích thước batch trong mỗi lần train
NW = 2 # số worker dùng để load dataset
EPOCHS = 3 # số epochs để huấn luyên mô hình
ACCUMULATE = 4 # chỉ số trong thuật toán tối ưu adam
DEFAULT_MODEL ='../input/huggingface-code-models/codebert-base' # đường dẫn đến mạng code-bert đã học sẵn 

In [ ]:
if not os.path.exists("./data"):
    os.mkdir("./data")

## Load data

Load 100 % số file dùng để train. Tham số `percent = 1` thể hiện điều đó.

**Trên thực tế**, nhóm chỉ thực hiện load 15 % lượng file để train (`percent = 0.15`) bởi vì giới hạn về mặt phần cứng trên Kaggle

In [ ]:
def read_notebook(path):
    '''
    path : path to notebook file .json
    return : dataframe of notebook. each row inform a cell in notebok
    '''
    return (
        pd.read_json(
            path,
            dtype={'cell_type': 'category', 'source': 'str'})
            .assign(id=path.stem)
            .rename_axis('cell_id')
    )

percent = 1
paths_train = list((data_dir / 'train').glob('*.json'))
paths_train = paths_train[:int(len(paths_train)*percent)]
print('Loading training data ...')
notebooks_train = [
    read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
]
df = (
    pd.concat(notebooks_train)
        .set_index('id', append=True)
        .swaplevel()
        .sort_index(level='id', sort_remaining=False)
)
df

# Preprocess, integrate, split data


## Integrate

In [ ]:
#test
pd.read_json('../input/AI4Code/train/8a2564b730a575.json',
             dtype={'cell_type': 'category', 'source': 'str'}
            ).assign(id="8a2564b730a575").rename_axis('cell_id')

In [ ]:
df_orders = pd.read_csv(
    data_dir / 'train_orders.csv',
    index_col='id',
    squeeze=True,
).str.split() 
# Split the string representation of cell_ids into a list

df_orders_ = df_orders.to_frame().join(
    df.reset_index('cell_id').groupby('id')['cell_id'].apply(list),
    how='right',
)
df_orders

## Preprocessing

Thêm thông tin về thứ tự các cell bằng cột `rank` và chuẩn hóa về khoảng `0-1` ở cột `pct_rank`.

In [ ]:
def get_ranks(base, derived):
    '''
    return a rank collumn for nb (notebook)
    nb : df of notebook
    cell_order: order of cell in notebook
    '''
    return [base.index(d) for d in derived]

ranks = {}
for id_, cell_order, cell_id in df_orders_.itertuples():
    ranks[id_] = {'cell_id': cell_id, 'rank': get_ranks(cell_order, cell_id)}
df_ranks = (
    pd.DataFrame
        .from_dict(ranks, orient='index')
        .rename_axis('id')
        .apply(pd.Series.explode)
        .set_index('cell_id', append=True)
)
df_ranks

In [ ]:
df_ancestors = pd.read_csv(data_dir / 'train_ancestors.csv', index_col='id')

In [ ]:
df = (
    df.reset_index()
    .merge(df_ranks, on=["id", "cell_id"])
    .merge(df_ancestors, on=["id"])
)
df["pct_rank"] = df["rank"] / df.groupby("id")["cell_id"].transform("count")

## Split data train, test, val

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

NVALID = 0.1  # size of validation set
splitter = GroupShuffleSplit(n_splits=1, test_size=NVALID, random_state=0)
train_ind, val_ind = next(splitter.split(df, groups=df["ancestor_id"]))
train_df = df.loc[train_ind].reset_index(drop=True)
val_df = df.loc[val_ind].reset_index(drop=True)

train_df_mark = train_df[train_df["cell_type"] == "markdown"].reset_index(drop=True)
val_df_mark = val_df[val_df["cell_type"] == "markdown"].reset_index(drop=True)


Lưu lại các file data sau khi split phòng trường hợp cần sử dụng lại

In [ ]:
train_df_mark.to_csv("./data/train_mark.csv", index=False)
val_df_mark.to_csv("./data/val_mark.csv", index=False)
val_df.to_csv("./data/val.csv", index=False)
train_df.to_csv("./data/train.csv", index=False)

In [ ]:
# preprocessing: clean code
def clean_code(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.replace('[', ' ').replace(']', ' ').replace('(', ' ').replace(')', ' ').replace('{', ' ').replace('}', ' ').replace('=', ' ').replace(',', ' ')
    text = text.lower()
    text = text.replace('_', '')
    text = text.replace('\n', ' ')
    text = text.replace('.', ' ')
    text = re.sub(r'".*"', ' ', text)
    text = re.sub(r"'.*'", ' ', text)
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    return text


## Features extraction

Thực hiện trích xuất các đặc trưng bao gồm:
- Số lượng code cell.
- Số lượng markdown cell.
- 20 code cell được lấy cách đều nhau.

In [ ]:
def sample_cells(cells, n):
    cells = [str(cell).replace("\\n", "\n") for cell in cells]
#     cells = [code_preprocessing(cell) for cell in cells]
    
    if n >= len(cells):
        return [cell[:200] for cell in cells]
    else:
        results = []
        step = len(cells) / n
        idx = 0
        while int(np.round(idx)) < len(cells):
            results.append(cells[int(np.round(idx))])
            idx += step
        assert cells[0] in results
        if cells[-1] not in results:
            results[-1] = cells[-1]
        return results

df = val_df
def get_features(df):
    features = dict()
    df = df.sort_values("rank").reset_index(drop=True)
    for idx,sub_df in tqdm(df.groupby("id")):
        features[idx] = dict()
        total_md = sub_df[sub_df['cell_type']=='markdown'].shape[0]
        code_sub_df = sub_df[sub_df['cell_type']=='code']
        total_code = code_sub_df.shape[0]
        codes = sample_cells(code_sub_df['source'].values,20)
        features[idx]['total_code']= total_code
        features[idx]['total_md'] = total_md
        features[idx]['codes'] = codes
    return features

val_fts = get_features(val_df)
train_fts = get_features(train_df)

#save file
json.dump(val_fts, open("./data/val_fts.json","wt"))
json.dump(train_fts, open("./data/train_fts.json","wt"))

# Define MarkdownModel class

Thứ tự các layer bao gồm:
- CodeBert
- Linear(769,1)
Linear layer có kích thước 769 vì có thêm 1 giá trị đặc trưng được thêm vào sau khi đi qua CodeBert.

In [ ]:
from bisect import bisect

import torch.nn.functional as F
import torch.nn as nn
import torch
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup, AutoConfig


class MarkdownModel(nn.Module):
    def __init__(self, model_path = DEFAULT_MODEL):
        super(MarkdownModel, self).__init__()
        self.config = AutoConfig.from_pretrained(model_path, output_hidden_states=True)
        self.model = AutoModel.from_pretrained(model_path)
#         self.dropout = nn.Dropout(p=0.1)
        self.top = nn.Linear(769, 1)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(self, ids, mask, fts):
        x = self.model(ids, mask)[0]
        x = torch.cat((x[:, 0, :], fts), 1)
        x = self.top(x)
        return x

# Define MarkdownDataset class

- `total_max_len` là kích thước tối đa của encode.
- `md_max_len là 

In [ ]:
class MarkdownDataset(Dataset):

    def __init__(self, df, total_max_len, md_max_len, fts, model_name_or_path = DEFAULT_MODEL):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.md_max_len = md_max_len
        self.total_max_len = total_max_len  # maxlen allowed by model config
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
        self.fts = fts

    def __getitem__(self, index):
        row = self.df.iloc[index]
        inputs = self.tokenizer.encode_plus(
            row.source,
            None,
            add_special_tokens=True,
            max_length=self.md_max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
        code_inputs = self.tokenizer.batch_encode_plus(
            [str(x) for x in self.fts[row.id]["codes"]],
            add_special_tokens=True,
            max_length=23,
            padding="max_length",
            truncation=True
        )
        n_md = self.fts[row.id]["total_md"]
        n_code = self.fts[row.id]["total_code"]
        if n_md + n_code == 0:
            fts = torch.FloatTensor([0])
        else:
            fts = torch.FloatTensor([n_md / (n_md + n_code)])

        ids = inputs['input_ids']
        for x in code_inputs['input_ids']:
            ids.extend(x[:-1])
        ids = ids[:self.total_max_len]
        if len(ids) != self.total_max_len:
            ids = ids + [self.tokenizer.pad_token_id, ] * (self.total_max_len - len(ids))
        ids = torch.LongTensor(ids)

        mask = inputs['attention_mask']
        for x in code_inputs['attention_mask']:
            mask.extend(x[:-1])
        mask = mask[:self.total_max_len]
        if len(mask) != self.total_max_len:
            mask = mask + [self.tokenizer.pad_token_id, ] * (self.total_max_len - len(mask))
        mask = torch.LongTensor(mask)

        assert len(ids) == self.total_max_len

        return ids, mask, fts, torch.FloatTensor([row.pct_rank])

    def __len__(self):
        return self.df.shape[0]

In [ ]:
# order_df = pd.read_csv(data_dir/ "train_orders.csv").set_index("id")
df_orders = pd.read_csv(
    data_dir / 'train_orders.csv',
    index_col='id',
    squeeze=True,
).str.split()

In [ ]:
# train_df_mark = pd.read_csv("./data/train_mark.csv").drop("parent_id", axis=1).dropna().reset_index(drop=True)
# train_fts = json.load(open("./data/train_fts.json"))
# train_df_mark = pd.read_csv("./data/val_mark.csv").drop("parent_id", axis=1).dropna().reset_index(drop=True)
# val_fts = json.load(open("./data/val_fts.json"))
# val_df = pd.read_csv("./data/val.csv")

# Define dataset instance

In [ ]:
train_ds = MarkdownDataset(
    df = train_df_mark,
    fts = json.load(open('./data/train_fts.json')),
    total_max_len=512,
    md_max_len= 64,
)
val_ds = MarkdownDataset(
    df = val_df_mark,
    fts = json.load(open('./data/val_fts.json')),
    total_max_len=512,
    md_max_len= 64,
)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NW,
                          pin_memory=False, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NW,
                        pin_memory=False, drop_last=False)


In [ ]:
def read_data(data):
    return tuple(d.cuda() for d in data[:-1]), data[-1].cuda()


def validate(model, val_loader):
    model.eval()

    tbar = tqdm(val_loader, file=sys.stdout)

    preds = []
    labels = []

    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            with torch.cuda.amp.autocast():
                pred = model(*inputs)

            preds.append(pred.detach().cpu().numpy().ravel())
            labels.append(target.detach().cpu().numpy().ravel())
    
    return np.concatenate(labels), np.concatenate(preds)

# Train cell

In [ ]:
# from torch.optim import Optimizer

def train(model, train_loader, val_loader, epochs):
#     wandb.init()
    np.random.seed(0)
    # Creating optimizer and lr schedulers
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
#     no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    num_train_optimization_steps = int(EPOCHS * len(train_loader) / ACCUMULATE)
    optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5,
                      correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
    
#     optimizer = bnb.optim.AdamW8bit(optimizer_grouped_parameters, lr=3e-5)
#     optimizer = BertAdam(optimizer_grouped_parameters,lr=2e-5,warmup=.1)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.05 * num_train_optimization_steps,
                                                num_training_steps=num_train_optimization_steps)  # PyTorch scheduler

    criterion = torch.nn.L1Loss()
    scaler = torch.cuda.amp.GradScaler()

    for e in range(epochs):
        model.train()
        tbar = tqdm(train_loader, file=sys.stdout)
        loss_list = []
        preds = []
        labels = []

        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            with torch.cuda.amp.autocast():
                pred = model(*inputs)
                loss = criterion(pred, target)
            scaler.scale(loss).backward()
            if idx % ACCUMULATE == 0 or idx == len(tbar) - 1:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()

            loss_list.append(loss.detach().cpu().item())
            preds.append(pred.detach().cpu().numpy().ravel())
            labels.append(target.detach().cpu().numpy().ravel())

            avg_loss = np.round(np.mean(loss_list), 4)

            tbar.set_description(f"Epoch {e + 1} Loss: {avg_loss} lr: {scheduler.get_last_lr()}")
            

        y_val, y_pred = validate(model, val_loader)
        val_df["pred"] = val_df.groupby(["id", "cell_type"])["rank"].rank(pct=True)
        val_df.loc[val_df["cell_type"] == "markdown", "pred"] = y_pred
        y_dummy = val_df.sort_values("pred").groupby('id')['cell_id'].apply(list)
        score = kendall_tau(df_orders.loc[y_dummy.index], y_dummy)
        print("Preds score", score)
        
        
        torch.save(model.state_dict(), "./model.bin")        
        del pred, inputs, target, loss, 
        gc.collect()
        torch.cuda.empty_cache()
        
    return model, y_pred


model = MarkdownModel()
model = model.cuda()
model, y_pred = train(model, train_loader, val_loader, epochs=EPOCHS)

In [ ]:
def predict(model_path, ckpt_path):
    model = MarkdownModel(model_path)
    model = model.cuda()
    model.eval()
    model.load_state_dict(torch.load(ckpt_path))
    BS = 32
    NW = 2
    MAX_LEN = 64
    test_df["pct_rank"] = 0
    test_ds = MarkdownDataset(test_df[test_df["cell_type"] == "markdown"].reset_index(drop=True), md_max_len=64,total_max_len=512, model_name_or_path=model_path, fts=test_fts)
    test_loader = DataLoader(test_ds, batch_size=BS, shuffle=False, num_workers=NW,
                              pin_memory=False, drop_last=False)
    _, y_test = validate(model, test_loader)
    return y_test

## Load test dataset

In [ ]:
paths_test = list((data_dir/'test').glob('*.json'))
notebook_test = [
    read_notebook(path) for path in paths_test
]
test_df = (
    pd.concat(notebook_test)
    .set_index('id',append=True)
    .swaplevel()
    .sort_index(level='id',sort_remaining=False)
).reset_index()
test_df["pct_rank"] = 0
test_df["rank"] = test_df.groupby(["id", "cell_type"]).cumcount()
test_df["pred"] = test_df.groupby(["id", "cell_type"])["rank"].rank(pct=True)
test_fts = get_features(test_df)

Tiến hành predict kết quả từ model vừa huấn luyện được

In [ ]:
model_path = "../input/huggingface-code-models/codebert-base"
ckpt_path = "./model.bin"
y_test = predict(model_path, ckpt_path)

## Save file submission

Tiến hành lưu lại file submission và đặt tên là **submission1** để phân biệt với file submission2 bên dưới

In [ ]:
test_df.loc[test_df["cell_type"] == "markdown", "pred"] = y_test
sub_df = test_df.sort_values("pred").groupby("id")["cell_id"].apply(lambda x: " ".join(x)).reset_index()
sub_df.rename(columns={"cell_id": "cell_order"}, inplace=True)
sub_df.to_csv("./submission1.csv", index=False)

# DistilBert baseline

# Ý tưởng: 
Sử dụng DistilBert để tokenize trong MarkdownDataset class và cấu hình lại Distilbert model cho phù hợp với bài toán trong MarkdownModel class.

**Lưu ý**: phần DistilBert có dùng lại một số hàm đã được code từ phần phía trên.

## Setup

In [ ]:
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn.metrics import mean_squared_error

## Config

In [ ]:
data_dir = Path('../input/AI4Code')
BERT_PATH = '../input/huggingface-bert-variants/distilbert-base-uncased/distilbert-base-uncased'

## Load data

Load toàn bộ file dùng để train (`LIMIT = None`)

**Trên thực tế**, nhóm đã load 100.000 file dùng để train (`LIMIT = 100000`) bởi vì giới hạn về mặt phần cứng trên Kaggle

In [ ]:
LIMIT = None
paths_train = list((data_dir / 'train').glob('*.json'))[:LIMIT]
notebooks_train = [
    read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
]
df = (
    pd.concat(notebooks_train)
    .set_index('id', append=True)
    .swaplevel()
    .sort_index(level='id', sort_remaining=False)
)
df

## Quan sát sơ lược dữ liệu vừa đọc được

In [ ]:
df_orders = pd.read_csv(
    data_dir / 'train_orders.csv',
    index_col='id',
    squeeze=True,
).str.split()
df_orders

Phần preprocessing bên dưới tương tự với phần preprocessing của mô hình bên trên (codebert)

In [ ]:
df_orders_ = df_orders.to_frame().join(
    df.reset_index('cell_id').groupby('id')['cell_id'].apply(list),
    how='right',
)
ranks = {}
for id_,cell_order,cell_id in df_orders_.itertuples():
    ranks[id_] = {'cell_id': cell_id,'rank': get_ranks(cell_order,cell_id)}
df_ranks = (
    pd.DataFrame
    .from_dict(ranks,orient='index' )
    .rename_axis('id')
    .apply(pd.Series.explode)
    .set_index('cell_id', append=True)
)
df_ranks

In [ ]:
df_ancestors = pd.read_csv(data_dir / 'train_ancestors.csv', index_col='id')

In [ ]:
df = df.reset_index().merge(df_ranks, on=["id", "cell_id"]).merge(df_ancestors, on=["id"])

In [ ]:
df["pct_rank"] = df["rank"] / df.groupby("id")["cell_id"].transform("count")

## Split data train, test, val

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
NVALID = 0.1 # size of validate
splitter = GroupShuffleSplit(n_splits=1,test_size=NVALID,random_state=212)

train_ind,val_ind = next(splitter.split(df,groups=df["ancestor_id"]))
train_df = df.loc[train_ind].reset_index(drop=True)
val_df = df.loc[val_ind].reset_index(drop=True)
val_df

In [ ]:
train_df_mark = train_df[train_df["cell_type"] == "markdown"].reset_index(drop=True)
val_df_mark = val_df[val_df["cell_type"] == "markdown"].reset_index(drop=True)

# MarkdownModel class

Thứ tự các layer bao gồm:
- DistilBertModel
- Linear(768, 64)
- Dropout 0.1
- Linear(64, 1)
- Sigmoid

In [ ]:
class MarkdownModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.distill_bert = DistilBertModel.from_pretrained(BERT_PATH)
        self.top1 = nn.Linear(768,64)
        self.top2 = nn.Linear(64,1)
        self.dropout = nn.Dropout(p=0.1)
    def forward(self,ids,mask):
        x = self.distill_bert(ids,mask)[0][:,0,:]
        x = self.top1(x)
        x = self.dropout(x)
        x = self.top2(x)
        x = torch.sigmoid(x)
        return x

# MarkdownDataset class

Sử dụng kết quả của DistilBert để tokenize dữ liệu với chiều dài tối đa là 128.

In [ ]:
class MarkdownDataset(Dataset):
    def __init__(self,df,max_len):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.max_len = max_len
        self.tokenizer = DistilBertTokenizer.from_pretrained('../input/huggingface-bert/bert-large-uncased')
    def __getitem__(self,index):
        row = self.df.iloc[index]
        inputs = tokenizer.encode_plus(
            row.source,
            add_special_tokens=True,
            max_length = 128,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
        ids = torch.LongTensor(inputs['input_ids'])
        mask = torch.LongTensor(inputs['attention_mask'])
        return ids,mask, torch.FloatTensor([row['pct_rank']])
    def __len__(self):
        return self.df.shape[0]
train_ds = MarkdownDataset(train_df_mark,max_len=128)
val_ds = MarkdownDataset(val_df_mark,max_len=128)
len(val_ds)

Train and test data loader.

In [ ]:
BATCH = 16
NW = 2

train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=NW,
                          pin_memory=False, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=BATCH, shuffle=True, num_workers=NW,
                          pin_memory=False, drop_last=False)

In [ ]:
# Learning rate at specific epochs
def adjust_lr(optimizer, epoch):
    if epoch < 1:
        lr = 5e-5
    elif epoch < 2:
        lr = 4e-5
    elif epoch < 5:
        lr = 3e-5
    else:
        lr = 2e-5

    for p in optimizer.param_groups:
        p['lr'] = lr
    return lr

# Define optimizer
def get_optimizer(net):
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=3e-4, betas=(0.9, 0.999),
                                 eps=1e-08)
    return optimizer

def read_data(data):
    return tuple(d.cuda() for d in data[:-1]), data[-1].cuda()
# Predicts and labels 
def validate(model, val_loader):
    model.eval()
    tbar = tqdm(val_loader, file=sys.stdout)
    
    preds = []
    labels = []

    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            pred = model(inputs[0], inputs[1])

            preds.append(pred.detach().cpu().numpy().ravel())
            labels.append(target.detach().cpu().numpy().ravel())
    
    return np.concatenate(labels), np.concatenate(preds)

## Train

Train mô hình với 3 epoch.

In [ ]:
def train(model, train_loader, val_loader, epochs):
    np.random.seed(0)
    # Optimizer defined above
    optimizer = get_optimizer(model)
    # Using L1 loss
    criterion = torch.nn.L1Loss()
    
    for e in range(epochs):   
        model.train()
        tbar = tqdm(train_loader, file=sys.stdout)
        
        lr = adjust_lr(optimizer, e)
        
        loss_list = []
        preds = []
        labels = []

        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            optimizer.zero_grad()
            pred = model(*inputs)
            loss = criterion(pred, target)
            loss.backward()
            optimizer.step()
            
            loss_list.append(loss.detach().cpu().item())
            preds.append(pred.detach().cpu().numpy().ravel())
            labels.append(target.detach().cpu().numpy().ravel())
            
            avg_loss = np.round(np.mean(loss_list), 4)

            tbar.set_description(f"Epoch {e+1} Loss: {avg_loss} lr: {lr}")
      
        y_val, y_pred = validate(model, val_loader)
        print(len(y_pred),len(y_val))

        val_df["pred"] = val_df.groupby(["id", "cell_type"])["rank"].rank(pct=True)
        val_df.loc[val_df["cell_type"] == "markdown", "pred"] = y_pred

        y_dummy = val_df.sort_values("pred").groupby('id')['cell_id'].apply(list)
        score = kendall_tau(df_orders.loc[y_dummy.index], y_dummy)

        print('score : ', score)

        output_model_file = f"./my_own_model_file_{e}_{np.round(score, 5)}.bin"
        model_to_save = model.module if hasattr(model, 'module') else model
        torch.save(model_to_save.state_dict(), output_model_file)
                        
        print("Validation MSE:", np.round(mean_squared_error(y_val, y_pred), 4))
        print()
        
        #optimize memory
        del inputs, target
        gc.collect()
        
    return model, y_pred

model = MarkdownModel()
model = model.cuda()
model, y_pred = train(model, train_loader, val_loader, epochs=3)

## Load test data set

Thực hiện load lại test data set, bởi vì lượng test data khá nhỏ và để đảm bảo file test không hề bị modify nên nhóm sẽ load lại test dataset

In [ ]:
paths_test = list((data_dir/'test').glob('*.json'))
notebook_test = [
    read_notebook(path) for path in paths_test
]
test_df = (
    pd.concat(notebook_test)
    .set_index('id',append=True)
    .swaplevel()
    .sort_index(level='id',sort_remaining=False)
).reset_index()
test_df

Dự đoán kết quả của tập test.

In [ ]:
# dummy collumns to fit MarkdownDataset
test_df["pct_rank"] = 0
test_df["rank"] = test_df.groupby(["id", "cell_type"]).cumcount()
test_df["pred"] = test_df.groupby(["id", "cell_type"])["rank"].rank(pct=True)
# create dataset interence and loader inference
test_ds = MarkdownDataset(test_df[test_df["cell_type"]=="markdown"].reset_index(drop=True),max_len=128)
test_loader = DataLoader(test_ds,batch_size=BATCH,shuffle=False,num_workers=NW,
                        pin_memory=False,drop_last=False)

y_test = validate(model,test_loader)[1]
test_df.loc[test_df["cell_type"]=="markdown","pred"] = y_test

Sắp xếp các cell theo thứ tự đã predict và lưu kết quả vào file `submission2.csv`.

In [ ]:
sub_df = test_df.sort_values("pred").groupby("id")["cell_id"].apply(lambda x: " ".join(x)).reset_index()
sub_df.rename(columns={"cell_id": "cell_order"}, inplace=True)
sub_df.head()

In [ ]:
sub_df.to_csv("submission2.csv", index=False)

# Ensemble

* Từ kết quả 2 model trên ta lấy trung bình vị trí (thứ tự cell trong trong notebook) theo tỉ lệ 0.25:0.75 giữa của 2 kết quả.
* Sau đó sắp xếp lại và lưu vào một file kết quả cuối cùng

In [ ]:
df_1 = pd.read_csv('submission2.csv')
df_2 = pd.read_csv('submission1.csv')

In [ ]:
new_samples = []
for sample_idx in range(len(df_1)):
    sample_1 = {k: v for v, k in enumerate(df_1.iloc[sample_idx]['cell_order'].split(' '))}
    sample_2 = {k: v for v, k in enumerate(df_2.iloc[sample_idx]['cell_order'].split(' '))}
    for key in sample_1: sample_1[key] = ( (sample_1[key] * 0.25) + (sample_2[key] * 0.75) )
    new_samples.append(' '.join([i[0] for i in list(sorted(sample_1.items(), key=lambda x:x[1]))]))
df_1['cell_order'] = new_samples

Lưu lại file submission hoàn chỉnh

In [ ]:
df_1.to_csv('submission.csv', index = False)

In [ ]:
df_1

# Kết quả trên Bảng xếp hạng

[Result link](https://drive.google.com/file/d/1CUsJVvr_YvC5yjIndCSzA-J0VgGrnRJi/view?usp=sharing)

[Result link (backup)](https://drive.google.com/file/d/16vpEZseVqGdHZsOnw3opQbNUjmR8iMBn/view?usp=sharing)

![image](https://drive.google.com/uc?export=view&id=1CUsJVvr_YvC5yjIndCSzA-J0VgGrnRJi)

# Nhận xét:
- Kết quả trên bảng xếp hạng tương đối ổn.
- Mặc dù kết quả ổn, tuy nhiên, cả 2 cách tiếp cận trên vẫn chưa tận dụng tối đa tính chất của các file code (code cell luôn đảm bảo thứ tự).
- Thời gian train các mô hình khá lâu, sử dụng tối đa thời gian và cấu hình cho phép của Kaggle (12h/1 lần train) vẫn chỉ có thể train được 1 phần nhỏ của dữ liệu.
- Tồn tại nhiều loại ngôn ngữ khác nhau trong dữ liệu, theo [discussion](https://www.kaggle.com/competitions/AI4Code/discussion/329783). Do đó việc train 1 phần nhỏ và không có sự phân loại có thể làm giảm sự chính xác.

# Tài liệu tham khảo:

- **[Stronger baseline with code cells](https://www.kaggle.com/code/suicaokhoailang/stronger-baseline-with-code-cells)** by [suicaokhoailang](https://www.kaggle.com/suicaokhoailang)
- **[AI4Code Pytorch DistilBert Baseline](https://www.kaggle.com/code/aerdem4/ai4code-pytorch-distilbert-baseline)** by [Ahmet Erdem](https://www.kaggle.com/aerdem4)
- **[Getting Started with AI4Code](https://www.kaggle.com/code/ryanholbrook/getting-started-with-ai4code)**